# Compress and Merge DataFrames
----

This notebook aims to clean, compress and merge the datasets from collected from <a href="https://www.consumerfinance.gov/data-research/hmda/historic-data/">Consumer Finance</a>.

Each dataset on average is 700MB and this inhibted team collaboration. To counteract this a single dataset was needed which complied with GitHub Enterprise's 100MB limitations.

In order to achieve this:
 - Data reduced to only the Metro Areas in Florida, New York, Texas and California
 - Removed Income Outliers. (Applicants under the 20<sup>th</sup> percentile and over the 90<sup>th</sup> percentile)
 - Compressed all datapoints to numeric key:value pairs
 - Used GZip compression to reduce the csv file from 612.9MB to 99.8MB

In [46]:
import pandas as pd
import numpy as np

import os
import json

## Function Definition Block
----

In [47]:
def latino_bin(value):
    if value == 'Hispanic or Latino': output = 1
    else: output = 0
    return output

In [48]:
def loan_approved_bin(value):
    accepted = ["Loan originated",
                "Application approved but not accepted",
                "Loan purchased by the institution"]
    if value in accepted:
        return 1
    return 0

In [50]:
def create_compression_dict(df, columns, compress=True):
    """Creates a dictionary of enumerated key:value pairs in a given Pandas DataFrame
    based on a list of columns. This can then be mapped to the dataframe to compress
    the datapoints to a numeric value or vice versa.
    
    - Function defaults to a compression format where the keys are the original data
      and the values are numbers.
    - if compress is False it returns a dictionary where the keys are numbers and the
      values are the original datapoints."""
    df_dict = {}
    for col in columns:
        column_dict = {}
        if compress:
            for value, key in enumerate(df[col].value_counts().index):
                column_dict[key] = value
        else:
            for value, key in enumerate(df[col].value_counts().index):
                column_dict[value] = key
        df_dict[col] = column_dict
    return df_dict

In [51]:
def compress_dataframe(df, dictionary):
    """Simple function to map the compression/decompression dictionary to a DF."""
    df = df.copy(deep=True)
    
    for col in dictionary.keys():
        df[col] = df[col].map(dictionary[col])
    return df

In [52]:
def read_and_merge_dataframes(directory):
    """Reads all ***FILES*** in a directory and attempts to read them as CSV.
    ***Will Break if Non-CSV files are in directory***
    Also culls data to:
     - Income over 20th percentile, under 90th percentile
     - cities based on MSAMD
     - Adds Latino Bin
     - Adds Approve Bin"""
    dataframes = []
    files = os.listdir(directory)
    for i, file in enumerate(files):
        file_path = f"./cfpb_data/{file}"
        print(f"Pulling file {i+1}/{len(files)}")
    
        #read csv, add state column, make a list of df's
        df = pd.read_csv(file_path, low_memory=False)
        
        # Limit incomes to >20% and <90%
        prcntl = df['applicant_income_000s'].quantile([0.20,0.90]).values
        df = df[(df['applicant_income_000s'] > prcntl[0]) &
                (df['applicant_income_000s'] < prcntl[1])]
        
        #limit cities
        cities = [36740.0, 27260.0, 33124.0, 45300.0, #Florida
                  31084.0, 36084.0, 41740.0, #California
                  19124.0, 12420.0, 26420.0, #Texas
                  35614.0, 35004.0] #New York
        df = df[df["msamd"].isin(cities)]
        
        #add approve bin
        df["approve_bin"] = df["action_taken_name"].map(lambda x: loan_approved_bin(x))
        
        #Make a column of Latino, not Latinto
        df["latino"] = df['applicant_ethnicity_name'].map(lambda x: latino_bin(x))
        
        dataframes.append(df)
    print("Merging DataFrames")
    return pd.concat(dataframes)

# Main Block
----
 1. Read and Merge DataFrames
 2. Reduce Features
 3. Create a Compression and Decompression Dictionary
 4. Output Compression and Decompression Dictionary
 5. Compress DataFrame
 6. Output Compressed DataFrame

### Read and Merge DataFrames
----
Initial size of all files in the folder were 13.3GB

In [53]:
merged_df = read_and_merge_dataframes("./cfpb_data/")

Pulling file 1/16
Pulling file 2/16
Pulling file 3/16
Pulling file 4/16
Pulling file 5/16
Pulling file 6/16
Pulling file 7/16
Pulling file 8/16
Pulling file 9/16
Pulling file 10/16
Pulling file 11/16
Pulling file 12/16
Pulling file 13/16
Pulling file 14/16
Pulling file 15/16
Pulling file 16/16
Merging DataFrames


### Reduce Features
----
In order to reduce the size some of the less relavant features should be cut. Based on Vadim's EDA the DataFrames can be reduced to the list below. Result is a reduction from 80 features to 52 features. 28 features across 4,715,850 rows should reduce the size of the file substantially.

In [54]:
merged_df.shape

(4715850, 80)

In [55]:
indices_list=['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr',
       'agency_code', 'loan_type_name', 'loan_type', 'loan_purpose_name',
       'loan_purpose', 'owner_occupancy_name', 'owner_occupancy', 'loan_amount_000s',
       'preapproval_name', 'preapproval', 'action_taken_name', 'action_taken',
       'msamd_name', 'msamd', 'state_name', 'county_name', 'census_tract_number',
       'applicant_ethnicity_name', 'applicant_ethnicity', 'co_applicant_ethnicity_name',
       'co_applicant_ethnicity', 'applicant_race_name_1', 'applicant_race_1',
       'co_applicant_race_name_1', 'co_applicant_race_1', 'applicant_sex_name',
       'co_applicant_sex_name', 'applicant_income_000s',
       'purchaser_type_name', 'purchaser_type', 'denial_reason_name_1',
       'denial_reason_1', 'denial_reason_name_2', 'denial_reason_2',
       'denial_reason_name_3', 'denial_reason_3', 'hoepa_status_name', 'hoepa_status',
       'lien_status_name', 'lien_status', 'population', 'minority_population',
       'hud_median_family_income', 'tract_to_msamd_income', 'number_of_owner_occupied_units',
       'number_of_1_to_4_family_units', "latino", "approve_bin"]

In [56]:
merged_df=merged_df[indices_list]

In [57]:
merged_df.shape

(4715850, 52)

## Create a Compression Dictionary
----

The datasets are massive. Simply reducing the features and keeping it in a zip is enough to get it to a size that we can put on GitHub. In order to reduce the file size further all values in the final dataset are going to be stored as numerical and translated back as needed.

### Create Dictionaries

In [58]:
# Deprecated. Even numerical columns needed to be compressed
categorical_features = merged_df.select_dtypes(exclude ="number").columns.tolist()

In [60]:
compression_dictionary = create_compression_dict(merged_df, merged_df.columns)
decompression_dictionary = create_compression_dict(merged_df, merged_df.columns, False)

### Output Compression and Decompression Dictionary
----

In [61]:
with open("json_files/compression_dictionary.json", "w") as json_file:
    json.dump(compression_dictionary, json_file)

with open("json_files/decompression_dictionary.json", "w") as json_file:
    json.dump(decompression_dictionary, json_file)

## Compress the file
----
Using GZip formatting reduced final CSV file from 612,926 KB to 99,841 KB.

In [62]:
merged_df = compress_dataframe(merged_df, compression_dictionary)

## Output Compressed DataFrame
----
End result is a 13,321% decrease in size from the original 13.3GB to the finalized 99.8MB

In [63]:
merged_df.to_csv("./merged_df/merged_df.csv.gz", index=False, compression="gzip")